# Advanced Querying Mongo

Importing libraries and setting up connection

In [43]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [44]:
from pymongo import MongoClient
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [45]:
cliente=MongoClient('mongodb://localhost:27017')

db=cliente.companies

In [46]:
collection = db.companies
type(collection)

pymongo.collection.Collection

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [47]:
# Your Code
list(collection.find({'name': 'Babelgum'},
                    {'_id':0,'name':1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [48]:
# Your Code
ejer2 =collection.find({'number_of_employees': {'$gte':5000}}, {'_id': False, 'name':True, 'number_of_employees': True}).sort('number_of_employees', -1).limit(20)
df2 = pd.DataFrame(ejer2)
df2.head()

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [49]:
# Your Code'
ejer3=collection.find({'founded_year': {'$gte':2000, '$lte':2005}  }, {'_id': False, 'name':True, 'founded_year': True})
df3 = pd.DataFrame(ejer3)
df3.head()

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [50]:
# Your Code
ejer4= collection.find({'$and':[{'ipo.valuation_amount': {'$gte':1e8}}, {'founded_year':{'$lte':2010}}]},  {'_id': False, 'name':True, 'ipo.valuation_amount': True})
df4=pd.json_normalize(data=ejer4, meta=['name', 'ipo.valuation_amount'])
df4.head()

,name,ipo.valuation_amount
0,Facebook,104000000000
1,Twitter,18100000000
2,Yelp,1300000000
3,LinkedIn,9310000000
4,Amazon,100000000000


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [51]:
# Your Code'
ejer5=collection.find({'$and':[{'number_of_employees': {'$lt':1e3}}, {'founded_year':{'$lte':2005}}]}, {'_id': False, 'name':True, 'number_of_employees': True, 'founded_year':True}).sort('number_of_employees', -1).limit(10)
df5 = pd.DataFrame(ejer5)
df5.head()

,name,number_of_employees,founded_year
0,Infinera Corporation,974,2000
1,Box,950,2005
2,NorthPoint Communications Group,948,1997
3,888 Holdings,931,1997
4,Forrester Research,903,1983


### 6. All the companies that don't include the `partners` field.

In [52]:
# Your Code
ejer6= collection.find({'partners': {'$ne': True}},{'_id': False, 'name':True})

df6 = pd.DataFrame(ejer6)
df6.head()


,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [53]:
# Your Code
ejer7= collection.find({'category_code': None},{'_id': False, 'name': True, 'category_code':True})

df7 = pd.DataFrame(ejer7)
df7.head()

,name,category_code
0,Collective,None
1,Snimmer,None
2,KoolIM,None
3,Level9 Media,None
4,VidKing,None


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [54]:
# Your Code
ejer8=collection.find({'number_of_employees': {'$gte':100, '$lt':1000}}, {'_id': False, 'name':True, 'number_of_employees': True})
df8 = pd.DataFrame(ejer8)
df8.head()

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [55]:
# Your Code
ejer9= collection.find({},{'_id': False, 'name':True, 'number_of_employees': True}).sort("ipo.valuation_amount",1) 
df9=pd.json_normalize(data=ejer9)
df9.head()


,name,number_of_employees
0,Wetpaint,47.0
1,AdventNet,600.0
2,Zoho,1600.0
3,Digg,60.0
4,Omnidrive,NaN


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [56]:
ejer10=collection.find({}, {'_id': False, 'name':True, 'number_of_employees': True}).sort('number_of_employees', -1).limit(10)
df10 = pd.DataFrame(ejer10)
df10.head()

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [57]:
# Your Code
ejer11=collection.find({'founded_month': {'$gte':6}}, {'_id': False, 'name':True, 'founded_month': True}).limit(1000)
df11 = pd.DataFrame(ejer11)
df11.head()

,name,founded_month
0,Wetpaint,10
1,Zoho,9
2,Digg,10
3,Omnidrive,11
4,Postini,6


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [60]:
# Your Code
ejer12=collection.find({'$and':[{'founded_year': {'$lt':2000}},{'acquisition.price_amount': {'$gte':1e9}}]}, {'_id': False, 'name':True, 'acquisition.price_amount':1})

df12=pd.json_normalize(data=ejer12, meta=['acquisition.price_amount'])
df12.head()

,name,acquisition.price_amount
0,PayPal,1500000000
1,Sun Microsystems,7400000000
2,Zappos,1200000000
3,Alibaba,1000000000
4,Sabre,4300000000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [72]:
# Your Code
ejer13=collection.find({'acquisition.acquired_year': {'$gte':2010}}, {'_id': False, 'name':True, 'acquisition.price_amount':1}).sort('acquisition.price_amount', -1)

df13=pd.json_normalize(data=ejer13, meta=['name','acquisition.price_amount'])
df13.head()

,name,acquisition.price_amount
0,T-Mobile,3.900000e+10
1,Goodrich Corporation,1.840000e+10
2,LSI,6.600000e+09
3,National Semiconductor,6.500000e+09
4,Sybase,5.800000e+09


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [75]:
# Your Code
ejer14=collection.find({'founded_year': {'$gte': 1 }}, {'_id': False, 'name':True, 'founded_year':1}).sort('founded_year', 1)

df14=pd.DataFrame(ejer14)
df14.head()

,name,founded_year
0,Alstrasoft,1800
1,SmallWorlds,1800
2,US Army,1800
3,DuPont,1802
4,McKesson,1833


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [78]:
# Your Code
ejer15=collection.find({'founded_day': {'$gte': 7 , '$lte': 17}}, {'_id': False, 'name':True, 'acquisition.price_amount':1}).sort('acquisition.price_amount', -1).limit(10)

df15=pd.json_normalize(ejer15)
df15.head()

,name,acquisition.price_amount
0,Yammer,1200000000
1,Yammer,1200000000
2,Adify,300000000
3,Zong,240000000
4,Topsy Labs,200000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [86]:
# Your Code
ejer16=collection.find({'$and':[{'category_code': 'web'}, {'number_of_employees':{'$gte': 4000}}]}, {'_id': False, 'name':True, 'number_of_employees':1}).sort('number_of_employees', -1)

df16=pd.DataFrame(ejer16)
df16.head()

,name,number_of_employees
0,Experian,15500
1,eBay,15000
2,Yahoo!,13600
3,Rakuten,10000
4,Los Angeles Times Media Group,10000


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [92]:
# Your Code
ejer17=collection.find({'$and':[{'acquisition.price_amount': {'$gte': 1e7}}, {'acquisition.price_currency_code' : 'EUR'}]}, {'_id': False, 'name':True, 'acquisition.price_amount':1})

df17=pd.json_normalize(ejer17)
df17.head()

,name,acquisition.price_amount
0,ZYB,31500000
1,Apertio,140000000
2,Greenfield Online,40000000
3,Webedia,70000000
4,Wayfinder,24000000


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [95]:
# Your Code acquired_month
ejer18=collection.find({'acquisition.acquired_month': {'$lte': 3}}, {'_id': False, 'name':True, 'acquisition.price_amount':1}).limit(10)

df18=pd.json_normalize(ejer18)
df18.head()

,name,acquisition.price_amount
0,Kyte,NaN
1,NetRatings,327000000.0
2,blogTV,NaN
3,Livestream,NaN
4,iContact,169000000.0


In [99]:
#Avoid NaN
df18 = df18.where(pd.notnull(df18), "")
df18.head()

,name,acquisition.price_amount
0,Kyte,
1,NetRatings,327000000.0
2,blogTV,
3,Livestream,
4,iContact,169000000.0


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [102]:
# Your Code
ejer19=collection.find({'$and':[{'founded_year': {'$gte': 2000, '$lte': 2010}}, {'acquisition.acquired_year':{'$lt':2011}}]}, {'_id': False, 'name':True, 'acquisition.acquired_year':1, 'founded_year':1})

df19=pd.json_normalize(ejer19)
df19.head()

,name,founded_year,acquisition.acquired_year
0,StumbleUpon,2002,2009
1,Gizmoz,2003,2009
2,Helio,2005,2008
3,Joost,2006,2009
4,Plaxo,2002,2008


### 20. All the companies that have been 'deadpooled' after the third year.

In [104]:
# Your Code
ejer20=collection.find({'deadpooled_year': {'$gt': 3}}, {'_id': False, 'name':True, 'deadpooled_year':1})

df20=pd.json_normalize(ejer20)
df20.head()

,name,deadpooled_year
0,Omnidrive,2008
1,Babelgum,2013
2,Sparter,2008
3,Thoof,2013
4,Mercora,2008
